In [1]:
from nltk.corpus import wordnet
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm, tqdm_notebook
import vsm
import data_loading
import nlu_utils


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Load the data
msr = data_loading.MSR()
dev = msr.dev()

In [53]:
gutenberg = msr.train_word_word_cooccurence(window=5, vocab_size=15000, load=True)
gutenberg.head()

Loading existing co-occurence matrix


,the,and,of,to,gonna,in,i,he,was,have,...,potter,hospitals,medal,truer,inkling,ether,disorders,paste,manufacturer,contractor
the,575465,773533,1152113,512908,273342,428605,190639,215837,283627,229895,...,60,96,80,41,54,107,139,59,102,116
and,773533,142551,366491,305755,260583,192941,141314,152640,154972,126417,...,54,76,49,31,14,36,51,58,34,30
of,1152113,366491,101784,186345,284284,176987,92880,93629,123952,102824,...,21,49,46,31,108,48,84,42,30,33
to,512908,305755,186345,90429,167945,104954,166567,140720,122609,141443,...,27,24,20,22,19,21,23,23,28,27
gonna,273342,260583,284284,167945,66256,153948,92434,99841,123110,96069,...,26,9,65,43,11,20,14,45,55,38


In [54]:
word_counts = msr.word_count()
word_counts

100%|██████████| 522/522 [03:53<00:00,  2.23it/s]


defaultdict(int,
            {'tom': 13984,
             'swift': 2285,
             'and': 1405243,
             'his': 425565,
             'submarine': 416,
             'boat': 5367,
             'or': 115460,
             'under': 25009,
             'the': 2408898,
             'ocean': 1519,
             'for': 305977,
             'sunken': 323,
             'treasure': 1520,
             'by': 173940,
             'victor': 1035,
             'appleton': 22,
             'contents': 1229,
             'i': 642889,
             'news': 3894,
             'of': 1214503,
             'gonna': 858519,
             'wreck': 646,
             'ii': 4046,
             'finishing': 383,
             'iii': 2408,
             'mr': 62416,
             'berg': 221,
             'is': 228598,
             'astonished': 1734,
             'iv': 2128,
             'imprisoned': 377,
             'v': 1676,
             'suspicious': 766,
             'vi': 1189,
             'turning': 596

In [32]:
# Calculate PPMI matrix
ppmi = vsm.pmi(gutenberg)
ppmi.head()

,the,and,of,to,gonna,in,i,he,was,have,...,strung,bounding,accomplishments,wee,inflict,denial,gratifying,arouse,clustered,acceptable
the,0.000000,0.095648,0.650427,0.0,0.000000,0.272979,0.00000,0.000000,0.047417,0.000000,...,0.052487,0.47098,0.000000,0.000000,0.000000,0.000000,0.000000,0.106181,0.654561,0.000000
and,0.095648,0.000000,0.016664,0.0,0.009150,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.321372,0.57140,0.418013,0.315610,0.000000,0.000000,0.000000,0.000000,0.501739,0.000000
of,0.650427,0.016664,0.000000,0.0,0.252597,0.056530,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.614017,0.000000,0.000000,0.567073,0.343913,0.000000,0.298760,0.000000
to,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.08887,0.053962,0.000000,0.109801,...,0.048240,0.00000,0.000000,0.000000,0.886804,0.000000,0.554045,1.075179,0.000000,0.744471
gonna,0.000000,0.009150,0.252597,0.0,0.000000,0.250607,0.00000,0.000000,0.214384,0.000000,...,0.377984,0.00000,0.000000,1.178747,0.235051,0.595952,0.008997,0.000000,0.041806,0.000000


In [55]:
dppmi = nlu_utils.dpmi(gutenberg)
dppmi.head()

,the,and,of,to,gonna,in,i,he,was,have,...,potter,hospitals,medal,truer,inkling,ether,disorders,paste,manufacturer,contractor
the,0.000000,0.083915,0.636722,0.0,0.000000,0.263833,0.000000,0.000000,0.041202,0.000000,...,0.028995,0.497975,0.319133,0.000000,0.000000,0.601737,0.840075,0.028987,0.551037,0.645720
and,0.083915,0.000000,0.003818,0.0,0.000837,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.428238,0.771578,0.338459,0.000000,0.000000,0.030284,0.349589,0.515447,0.000000,0.000000
of,0.636722,0.003818,0.000000,0.0,0.242311,0.046271,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.487586,0.427228,0.000000,1.223837,0.463122,0.997320,0.347878,0.000134,0.059879
to,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.096984,0.058426,0.000000,0.113984,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031093,0.000000
gonna,0.000000,0.000837,0.242311,0.0,0.000000,0.244879,0.000000,0.000000,0.211588,0.000000,...,0.190689,0.000000,1.102653,0.641932,0.000000,0.000000,0.000000,0.746212,0.926521,0.526507


In [63]:
class PPMIModel:
    def __init__(self, pmi, word_counts=word_counts, verbose=False):
        self.pmi = pmi
        self.index_to_label = ['a', 'b', 'c', 'd', 'e']
        self.verbose = verbose
        self.word_counts = word_counts
    
    def answer(self, problem):
        scores = []
        question = problem['question']
        scores.append(self.score(question, problem['a)'])) 
        scores.append(self.score(question, problem['b)'])) 
        scores.append(self.score(question, problem['c)'])) 
        scores.append(self.score(question, problem['d)'])) 
        scores.append(self.score(question, problem['e)']))
        return self.index_to_label[np.argmax(scores)], scores
    
    def approx_ppmi(self, proposal_token, proposal_synonyms, word_token):
        pos = nlu_utils.spacy_to_wn_tag(word_token.pos_)
        word_synonyms = nlu_utils.get_alternate_words(word_token.norm_, pos, self.word_counts)
        # try matching using different versions of the proposal word
        for psyn in proposal_synonyms:
            score = self.ppmi(psyn, word_token.norm_)
            if score is not None:
                if self.verbose:
                    print("Used synonym for proposal word: {} -> {}".format(proposal_token.text, psyn))
                return score
        # try matching using different versions of the non-proposal word
        for wsyn in word_synonyms:
            score = self.ppmi(wsyn, proposal_token.norm_)
            if score is not None:
                if self.verbose:
                    print("Used synonym: {} -> {}".format(word_token.text, wsyn))
                return score
        # Next just try all combos
        for psyn in proposal_synonyms:
            for wsyn in word_synonyms:
                score = self.ppmi(psyn, word_token.norm_)
                if score is not None:
                    if self.verbose:
                        print("Used synonym: {} -> {} and {} -> {}".format(proposal_token.text, psyn, word_token.text, wsyn))
                    return score
        if self.verbose:
            print("UNABLE TO FIND ANY SYNONYMS IN VOCABULARY")
        return None

    def ppmi(self, proposal, word):
        try:
            return self.pmi.loc[proposal, word]
        except KeyError:
            return None

    def substitute(self, sentence, proposal):
        sentence_list = sentence.split()
        i = sentence_list.index('_____')
        sentence_list[i] = proposal
        return ' '.join(sentence_list)
    
    def featurize(self, doc, proposal_token):
        features = []
        # Remove determiners, coordinating conjunctions, pronouns, add everything else to feature set
        for token in doc:
            if token.pos_ not in ['DET', 'PRON', 'CCONJ']:
                features.append(token)
        # Add dependencies to the feature set again (effectively increase their weight)
        features += nlu_utils.get_ancestors_of_word(proposal_token)
#         features += nlu_utils.get_children_of_word(proposal_token)
        return features        

    def score(self, sentence, proposal):
        full_sentence = self.substitute(sentence, proposal)
        doc = nlu_utils.get_spacy_doc(full_sentence)
        _, proposal_token = nlu_utils.get_token(doc, proposal)
        features = self.featurize(doc, proposal_token)

        pos = nlu_utils.spacy_to_wn_tag(proposal_token.pos_)
        synonyms = nlu_utils.get_alternate_words(proposal_token.norm_, pos, self.word_counts)

        tot_score = 0
        for token in features:
            if token == proposal_token:
                continue
            if token.is_punct or token.is_space:
                continue
            score = self.ppmi(proposal_token.norm_, token.norm_)
            if score is None:
                score = self.approx_ppmi(proposal_token, synonyms, token)
            tot_score += score if score is not None else 0
        return tot_score

In [64]:
model = PPMIModel(dppmi, word_counts=word_counts, verbose=False)
print("Making predictions")
predictions = []
scores = []
for i, (_, problem) in enumerate(dev.iterrows()):
    ans, sc = model.answer(problem)
    predictions.append(ans)
    scores.append(sc)
    if i % 200 == 1:
        print("------------------------------------------------------------------------------------------------------")
        print(problem['question'])
        print(problem)
        print("prediction: {}".format(ans))
        print(sc)

print("DONE")

Making predictions
------------------------------------------------------------------------------------------------------
It was only after a painful and prolonged scene that she was _____ by the butler and the footman.
question    It was only after a painful and prolonged scen...
a)                                                    ejected
b)                                                   consumed
c)                                                   awakened
d)                                                   startled
e)                                                   softened
answer                                                      a
Name: 727, dtype: object
prediction: c
[2.2147216425728047, 3.7990566115800215, 9.552362924032323, 7.314713706857974, 3.9896505021310475]
------------------------------------------------------------------------------------------------------
We had reached Pall Mall as we _____ , and were walking down it from the St. James's end.
question    We 

In [65]:
print(accuracy_score(dev.loc[:, 'answer'], predictions))

0.5352564102564102


#### Tuning Stop Word Removal:
* Baseline: 0.5272435897435898
* Removing DET: 0.530448717948718 (Helpd)
* Removing CCONJ: 0.5256410256410257 (Hurt)
* Removing PRON: 0.5288461538461539 (Barely Helped)
* Removing PROPN: 0.5272435897435898 (No effect)
* Removing AUX: 0.5272435897435898 (No effect)
* Removing DET + PRON: 0.5336538461538461 (Helped more)
* REMOVING DET + PRON + CCONJ: 0.5368589743589743 (Helped even more)

#### Tuning Dependency Parsing Features
* Baseline: 0.5272435897435898
* Ancestors ONLY: 0.36698717948717946
* Children ONLY: 0.28846153846153844
* Ancestors and Children ONLY: 0.4310897435897436
* Ancestors and Children plus Sentence: 0.5064102564102564 (Hurt)
* Ancestors plus Sentence: 0.5256410256410257 (Slightly hurt)
* Children plus Sentence: 0.4951923076923077 (Hurt)

## Results
* BASELINE: Window=5, Vocab=10000, synonyms=True: 0.482371794872
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true: 0.5336538461538461
* Window=5, Vocab=12500, synonyms_POS=True, spacy.norm_=true: 0.5464743589743589  **
* Window=5, Vocab=12500, synonyms_POS_ordered=True, spacy.norm_=true: 0.5368589743589743  ```??? Why is this lower?```
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true, remove POS stopwords: 0.530448717948718
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true, remove POS stopwords, use children and ancestors: 0.5256410256410257
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true, use children and ancestors: 0.5144230769230769
* Window=5, Vocab=15000, synonyms_POS_ordered=True, spacy.norm_=true, use ancestors: 0.5256410256410257
* Window=5, Vocab=15000, synonyms_POS_ordered=True, spacy.norm_=true, remove POS stopwords: 0.5368589743589743

### Hyperparameter tuning
* Window=5, Vocab=15000, synonyms_POS_ordered=True, spacy.norm_=true, remove POS stopwords, dppmi: 0.5384615384615384
* Window=10, Vocab=10000, synonyms_POS_ordered=True, spacy.norm_=true, remove POS stopwords, dppmi: 0.5384615384615384

In [24]:
def top_2_accuracy_score(scores, answers):
    index_to_label = ['a', 'b', 'c', 'd', 'e']
    n_correct = 0
    for i, (_, ans) in enumerate(answers.iteritems()):
        pred1, pred2 = np.argsort(-np.array(scores[i]))[:2]
        pred1 = index_to_label[pred1]
        pred2 = index_to_label[pred2]
        if pred1 == ans or pred2 == ans:
            n_correct += 1
    return n_correct / len(answers)
        
top_2_accuracy_score(scores, dev.loc[:, 'answer'])

0.7051282051282052

In [28]:
def examine_model(predictions, scores):
    for i, (_, problem) in enumerate(dev.iterrows()):
        ans_to_index = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}
        total_diff = 0.0
        total_incorrect = 0
        if predictions[i] != problem['answer']:
            print("--------------------------------------------------------------------------------------------------------------------------------")
            print(problem['question'])
            doc = nlu_utils.get_spacy_doc(problem['question'])
            print(' '.join([t.norm_ for t in doc]))
            print(problem)
            print(predictions[i])
            print(scores[i])
            total_incorrect += 1
            total_diff += (scores[i][ans_to_index[predictions[i]]] - scores[i][ans_to_index[problem['answer']]])
    return total_diff / total_incorrect
        
        
examine_model(predictions, scores)

--------------------------------------------------------------------------------------------------------------------------------
His hair and whiskers were shot with gray , and his face was all crinkled and _____ like a withered apple.
his hair and whiskers were shot with gray , and his face was all crinkled and _ _ _ _ _ like gonna withered apple .
question    His hair and whiskers were shot with gray , an...
a)                                                 chattering
b)                                                picturesque
c)                                                    hopeful
d)                                                   puckered
e)                                                 glistening
answer                                                      d
Name: 97, dtype: object
e
[4.343345254275475, 0.41517889750040926, 1.184547060078838, 0.0, 13.894752805292605]
-------------------------------------------------------------------------------------------------------

Name: 705, dtype: object
c
[9.242933585235814, 9.417329605626954, 12.454334973099146, 8.087570392788363, 5.971700858882075]
--------------------------------------------------------------------------------------------------------------------------------
Who were these German people , and what were they doing living in this strange , _____ place.
who were these german people , and what were they doing living in this strange , _ _ _ _ _ place .
question    Who were these German people , and what were t...
a)                                                 infallible
b)                                                 impossible
c)                                             out-of-the-way
d)                                                 unladylike
e)                                                   untimely
answer                                                      c
Name: 246, dtype: object
e
[0, 0.029562344359216355, 1.4628725961062337, 0, 3.7957030613146667]
------------------------

there were two guides given us to _ _ _ _ _ with , an oak and an elm .
question    There were two guides given us to _____ with ,...
a)                                                      argue
b)                                                    quarrel
c)                                                      start
d)                                                       dine
e)                                                      lodge
answer                                                      c
Name: 566, dtype: object
e
[2.537116453378212, 4.048794562284839, 2.0650589718067, 3.0697737552109197, 4.555863782535121]
--------------------------------------------------------------------------------------------------------------------------------
I did not know about a projected _____ between herself and her husband.
i do not know about gonna projected _ _ _ _ _ between herself and her husband .
question    I did not know about a projected _____ between...
a)                            

i was just _ _ _ _ _ whether i should run for it , or whether i should perch behind her landau when gonna cab came through the street .
question    I was just _____ whether I should run for it ,...
a)                                                 commencing
b)                                                   learning
c)                                                    hearing
d)                                                    wishing
e)                                                  balancing
answer                                                      e
Name: 405, dtype: object
a
[3.951148591898751, 0.6388707739652753, 2.5769971250668227, 2.294651948911918, 0.833825725241943]
--------------------------------------------------------------------------------------------------------------------------------
I had the hint from Holmes that this smooth-faced pawnbroker's assistant was a _____ man a man who might play a deep game.
i have the hint from holmes that this smooth - faced 

i put myself in the man 's place and , having first gaged his intelligence , i try to _ _ _ _ _ how i should myself have proceeded under the same circumstances .
question    I put myself in the man's place and , having f...
a)                                                    witness
b)                                                    realize
c)                                                     forget
d)                                                    imagine
e)                                                      prove
answer                                                      d
Name: 962, dtype: object
b
[3.2822302537308525, 6.1204860456178, 5.4281277749590515, 4.602760698374134, 5.091710084314082]
--------------------------------------------------------------------------------------------------------------------------------
During two years I have had three _____ and one small job , and that is absolutely all that my profession has brought me.
during two years i have have t

...

again and again i have taken gonna problem to him , and have received an _ _ _ _ _ which has afterwards proved to be the correct one .
question    Again and again I have taken a problem to him ...
a)                                                arrangement
b)                                                 affliction
c)                                                 invitation
d)                                               entanglement
e)                                                explanation
answer                                                      e
Name: 48, dtype: object
c
[2.8503633226997436, 1.68062209530981, 5.578056449359339, 1.9647201386863178, 4.5203812089900275]
--------------------------------------------------------------------------------------------------------------------------------
It was the clank of the levers and the swish of the _____ cylinder.
it was the clank of the levers and the swish of the _ _ _ _ _ cylinder .
question    It was the clank of the l

admiration was , i _ _ _ _ _ , the first impression .
question    Admiration was , I _____ , the first impression.
a)                                                    obeyed
b)                                                   shouted
c)                                                    repeat
d)                                                    gasped
e)                                                    argued
answer                                                     c
Name: 202, dtype: object
a
[0.6623079760211656, 0.3928977653950495, 0.291055951915418, 0.0, 0.31971223519541314]
--------------------------------------------------------------------------------------------------------------------------------
I have tried to _____ it from the measurements.
i have tried to _ _ _ _ _ it from the measurements .
question    I have tried to _____ it from the measurements.
a)                                                recommend
b)                                                    fl

the furniture was scattered about in every direction , with dismantled shelves and open drawers , as if the lady have hurriedly _ _ _ _ _ them before her flight .
question    The furniture was scattered about in every dir...
a)                                                     taught
b)                                                   assigned
c)                                                  ransacked
d)                                                     warned
e)                                                    rebuked
answer                                                      c
Name: 783, dtype: object
d
[1.7397822831339051, 1.2853035165985776, 2.271261850915231, 2.889517562087098, 1.2345029027485686]
--------------------------------------------------------------------------------------------------------------------------------
I fainted when it was done , and I think that I must have been _____ for a long time.
i fainted when it was done , and i think that i must have been

altogether , look as i would , there was nothing remarkable about the man save his blazing red head , and the expression of _ _ _ _ _ chagrin and discontent upon his features .
question    Altogether , look as I would , there was nothi...
a)                                                  impatient
b)                                                    mocking
c)                                                  perpetual
d)                                                   reckless
e)                                                    extreme
answer                                                      e
Name: 332, dtype: object
b
[2.7963852309721196, 5.974173981735219, 3.7439347827595304, 3.0512143213149976, 4.159799278285586]
--------------------------------------------------------------------------------------------------------------------------------
It contained a loaf of bread , a tinned tongue , and two _____ of preserved peaches.
it contained gonna loaf of bread , gonna tinned t

1.6199732462098595